<a href="https://colab.research.google.com/github/bonillahermes/Hermes_Projects/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Sentimientos en Videos de YouTube en Español: Un Enfoque Integral Utilizando NLP y Modelos de Transcripción de Audio

## Introducción
Este proyecto tiene como objetivo realizar un análisis de sentimientos sobre el contenido de videos de YouTube en español. Utilizando diversas herramientas y bibliotecas de procesamiento de lenguaje natural (NLP), el proyecto descarga un video de YouTube, extrae el audio, transcribe el audio a texto y luego analiza el sentimiento del texto resultante. El análisis de sentimientos proporciona una comprensión de la tonalidad general del contenido del video, clasificándolo como positivo, negativo o neutral.

## Descripción del Código y su Uso
El código se divide en varias secciones, cada una de las cuales utiliza diferentes bibliotecas para completar tareas específicas:

### Descarga y Procesamiento de Videos:

- pytube y yt_dlp se utilizan para descargar videos de YouTube.
- subprocess y os se utilizan para manejar el procesamiento de archivos y directorios.

### Transcripción de Audio:

- whisper de OpenAI se utiliza para transcribir el audio del video a texto.

### Procesamiento de Lenguaje Natural (NLP):

- nltk y spacy se utilizan para tokenizar el texto, eliminar palabras vacías (stop words) y procesar el texto.
- Counter de la colección estándar de Python se utiliza para contar la frecuencia de las palabras.
- TfidfVectorizer se utiliza para el análisis temático del texto.

### Análisis de Sentimientos:

- transformers de Hugging Face se utiliza para analizar el sentimiento del texto transcrito, clasificando cada fragmento de texto como positivo, negativo o neutral.

### Visualización:

- matplotlib.pyplot y WordCloud se utilizan para crear visualizaciones del análisis, como nubes de palabras y gráficos de barras.

In [ ]:
# Instalar Whisper, ffmpeg, yt-dlp, pytube, nltk, spacy, y transformers
!pip install git+https://github.com/openai/whisper.git
!pip install pytube
!pip install yt-dlp --upgrade
!pip install nltk spacy transformers
!sudo apt-get install ffmpeg

# Descargar modelos de spaCy para inglés y español
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm


In [ ]:
# Librerías para descarga y procesamiento de videos
from pytube import YouTube
import subprocess
import yt_dlp
import os

# Librerías para transcripción de audio
import whisper

# Librerías para procesamiento de lenguaje natural (NLP)
import nltk
import spacy
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

# Librerías para análisis de sentimientos
from transformers import pipeline

# Librerías para visualización
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Otras utilidades
import string


In [ ]:
def download_youtube_video(url, output_path):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

video_url = "https://www.youtube.com/watch?v=-IdmQoLnlS0"  # Reemplaza con tu URL de YouTube
download_youtube_video(video_url, "/content")

# Identificar el archivo de audio descargado

audio_path = None
for file in os.listdir("/content"):
    if file.endswith(".wav"):
        audio_path = os.path.join("/content", file)
        break

if audio_path:
    print(f"Audio descargado en: {audio_path}")
else:
    print("No se pudo encontrar el archivo de audio.")


In [ ]:
# Cargar el modelo de Whisper
if audio_path:
    # Cargar el modelo de Whisper
    model = whisper.load_model("base")

    # Transcribir el audio
    result = model.transcribe(audio_path)
    text = result["text"]
    print(text)
else:
    print("No se pudo procesar la transcripción porque el archivo de audio no fue encontrado.")



### Resultado

Gobernos de todo el mundo dudan de los resultados de la elección presidencial del domingo en Venezuela, en la que el presidente Nicolás Maduro fue declarado relegido por la autoridad electoral venezolana. Estados Unidos, Brasil y Colombia cuestionaron el escrutinio que torga Maduro un tercer mandato de seis años, por 51% de los votos, frente a un 44% del candidato de la leaza opositora Edmundo González, Rusia. Nos preocupa seriamente que el resultado anunciado no refleje la voluntad ni los votos del pueblo venezolano, es fundamental que cada voto secuente de manera justa y transparente, que los funcionarios electorales compartan de inmediato la información con la oposición y los observadores independientes sin demora, y que las autoridades electorales publican el recuento detallado de los votos. El gobierno de Gustavo Petro, primer presidente de izquierda de la historia de Colombia y aléado de Maduro, pidió por su parte el conteto total de los votos, su verificación y auditoría de carácter independiente. Lo mismo pidió el presidente de Paraguay, Santiago Peña, quien mencionó los contos de la oposición, que difieren del resultado anunciado por el Consejo Nacional Electoral de Venezuela. Hay un reclamo de la oposición que dicen que de acordadas actas que ellos tienen de todas las mesas, ellos tenían una victoria del 70%, pero eso contradice la versión oficial. Brasília llamó una verificación imparcial de los resultados y Panamá retiro a su personal diplomático de Caracas y puso en sus pensos las relaciones bilaterales mientras que el presidente Chileno Gabriel Borich dijo que los resultados son difíciles de creer. Tienen que ser verificadas por verdores internacionales no dependientes ni partidarios del gobierno. Francia España, la Unión Europea, pidieron total transparencia sobre el proceso y los gobiernos de Argentina, Costa Rica, Ecuador, Guatemala, Perú, República Dominicana y Uruguay exigieron la revisión completa de los resultados. Condenar el fraude electoral perpetrado por el régimen del dictador Nicolás Maduro en la República Bolivarena de Venezuela, Argentina exige total transparencia del recuento de votos, no vamos a convalidar ningún resultado sin el respaldo de los veidores internacionales. El Consejo Permanente de la Organización de Estados Americanos convocó para el miércoles una reunión extraordinaria para abordar los resultados del proceso electoral. Venezuela anunció el retiro de su personal diplomático de Argentina, Chile, Costa Rica, Panamá, Perú, República Dominicana y Uruguay, ante lo que considero ingerensistas acciones de esos países. También pidió que se retiren los diplomáticos de esas naciones acreditados en caracas. China, Rusia, Cuba, Nicaragua, Honduras y Bolivia felicitaron a Maduro y México dijo que reconocer al resultado de las autoridades electorales de Venezuela. Le está dando el trunfo al presidente Maduro.


In [ ]:
# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load('es_core_news_sm')

# Transcribir el audio
text = result["text"]

# Tokenización
tokens = nltk.word_tokenize(text)

# Filtrar stop words y puntuación
stop_words = set(stopwords.words('spanish'))
filtered_tokens = [word for word in tokens if word.lower() not in stop_words and word not in string.punctuation]

# Procesamiento con spaCy
doc = nlp(' '.join(filtered_tokens))

In [ ]:
# Descargar recursos de NLTK
nltk.download('punkt')
nlp = spacy.load('es_core_news_sm')

# Tokenización
tokens = nltk.word_tokenize(text)

# Procesamiento con spaCy
doc = nlp(text)

In [ ]:
# Frecuencia de palabras
word_freq = Counter(filtered_tokens)
print("Frecuencia de palabras:", word_freq.most_common(10))

In [ ]:
# Entidades nombradas
print("Entidades nombradas:")
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
# Función para dividir el texto en oraciones
def split_into_sentences(text):
    return nltk.sent_tokenize(text)

# Análisis de sentimientos con modelo específico para español
sentiment_analysis = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Función para mapear etiquetas de estrellas a sentimientos
def map_sentiment(label):
    if label in ['1 star', '2 stars']:
        return 'negativo'
    elif label == '3 stars':
        return 'neutral'
    elif label in ['4 stars', '5 stars']:
        return 'positivo'
    return 'desconocido'

# Dividir el texto en oraciones y analizar cada oración
sentiments = []
for sentence in split_into_sentences(text):
    sentiment = sentiment_analysis(sentence)
    sentiment_label = sentiment[0]['label']
    sentiment_score = sentiment[0]['score']
    sentiment_mapped = map_sentiment(sentiment_label)
    sentiments.append((sentence, sentiment_label, sentiment_score, sentiment_mapped))

# Mostrar los resultados
print("Análisis de Sentimientos por Oración:")
for i, (sentence, label, score, mapped) in enumerate(sentiments):
    print(f"Oración {i + 1}: {sentence}")
    print(f"Sentimiento: {mapped} (Label: {label}, Score: {score})\n")

In [ ]:
# Crear listas de sentimientos y puntuaciones
sentiment_labels = [mapped for _, _, _, mapped in sentiments]
scores = [score for _, _, score, _ in sentiments]

# Contar las ocurrencias de cada sentimiento
sentiment_counts = Counter(sentiment_labels)

# Graficar los resultados
labels = sentiment_counts.keys()
sizes = sentiment_counts.values()

plt.figure(figsize=(8, 5))
plt.bar(labels, sizes, color=['red', 'yellow', 'green'])
plt.xlabel('Sentimientos')
plt.ylabel('Frecuencia')
plt.title('Frecuencia de Sentimientos en el Discurso')
plt.show()

## Interpretación de Resultados
- Sentimiento Predominante: La mayoría de las oraciones son clasificadas como negativas, lo cual refleja un sentimiento general desfavorable en el discurso analizado.
- Sentimientos Positivos: Hay pocas oraciones clasificadas como positivas, indicando momentos aislados de positividad o neutralidad.

## Conclusión
Este proyecto proporciona un enfoque integral para analizar los sentimientos de los videos de YouTube en español. Utilizando una combinación de herramientas y técnicas de procesamiento de lenguaje natural, el código permite descargar, transcribir y analizar el contenido de los videos, ofreciendo una visión de los sentimientos predominantes en el discurso analizado.